# Run on Google Colab

https://colab.research.google.com/github/Kapil3003farmonaut/Api/blob/main/api_endpoints.ipynb


# Start


In [2]:
import requests
import pandas as pd


UID = ""  # API testing

# Check Usage Details


In [ ]:
def get_my_usage(UID):

    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getMyUsage"
    bodyObj = {
        "UID": UID,
    }

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response.json


response = get_my_usage(UID)
print(response.json())

## Add/Submit field


In [ ]:
def submit_field(UID, CropCode, FieldName, PaymentType, PointCoordinates):
    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/submitField"
    bodyObj = {
        "UID": UID,
        "CropCode": CropCode,
        "FieldName": FieldName,
        "PaymentType": PaymentType,
        "Points": PointCoordinates,
    }

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response


CropCode = "1r"
FieldName = "My Test Field Kapil 22"
PaymentType = 1
PointCoordinates = [
    [78.47214764936771, 17.410292565471753],
    [78.47341901644077, 17.41025161622103],
    [78.47430414541569, 17.409652732380625],
    [78.47315615995731, 17.409586189610526],
]

response = submit_field(UID, CropCode, FieldName, PaymentType, PointCoordinates)
response.json()["FieldID"]

# Delete Field


In [ ]:
from json.decoder import JSONDecodeError


def delete_field(UID, FieldID):
    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/deleteField"
    bodyObj = {"UID": UID, "FieldID": FieldID}

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    try:
        json_data = response.json()
        print(json_data)
    except (AttributeError, JSONDecodeError) as e:
        print("Error occurred:", e)


FieldID = "1718261273888"
delete_field(UID, FieldID)

# Get All Fields/Farmers Data


In [ ]:
def get_all_feilds_data(UID):
    endpointUrl = (
        "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getAllFarmersData"
    )
    bodyObj = {
        "UID": UID,
    }

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response


all_fields_data = get_all_feilds_data(UID)


try:
    json_data = all_fields_data.json()
    if json_data:
        print("JSON data received.")
        # Further processing of the JSON data can be done here
    else:
        print("No JSON data received.")
except ValueError:
    print("Invalid JSON data received or no JSON data present.")


data_all_Field = []
for fieldId, fieldDetails in all_fields_data.json().items():
    # print(fieldId)
    lat_lon_pairs = []

    # Define custom sorting order
    def custom_sort(key):
        if key == "a":
            return 0
        elif key.startswith("P_"):
            return int(key[2:])
        else:
            return float("inf")  # Place any other keys at the end

    # Sort the keys of the Coordinates dictionary using the custom sorting function
    sorted_coordinates = sorted(fieldDetails["Coordinates"].keys(), key=custom_sort)

    # Iterate over the sorted keys
    for point in sorted_coordinates:
        coordinates = fieldDetails["Coordinates"][point]
        latitude = float(coordinates["Latitude"])
        longitude = float(coordinates["Longitude"])
        lat_lon_pairs.append([latitude, longitude])

    # for point, coordinates in fieldDetails["Coordinates"].items():
    #     latitude = float(coordinates["Latitude"])
    #     longitude = float(coordinates["Longitude"])
    #     lat_lon_pairs.append([latitude, longitude])

    data_all_Field.append(
        {
            "FieldID": fieldId,
            # "FieldAddress": fieldDetails["FieldAddress"],
            "FieldArea": fieldDetails["FieldArea"],
            "FieldMaxLat": fieldDetails["FieldMaxLat"],
            "FieldMaxLong": fieldDetails["FieldMaxLong"],
            "FieldMinLat": fieldDetails["FieldMinLat"],
            "FieldMinLong": fieldDetails["FieldMinLong"],
            # "hUnits": fieldDetails["hUnits"],
            "coodfinates": lat_lon_pairs,
        }
    )

df_field_data = pd.DataFrame(data_all_Field)
df_field_data

# Get Single Fields Data


In [ ]:
def get_single_field_data(UID, FieldID):
    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getFarmerData"
    bodyObj = {"UID": UID, "FieldID": FieldID}

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response


FieldID = "1718261488045"
field_data = get_single_field_data(UID, FieldID)
field_data.json()

# Sensed Days - Find when satellite data is available :dates


In [ ]:
def get_sesnsed_days(UID, FieldID):
    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getSensedDays"
    bodyObj = {
        # "PolygonID" : "1668483636222",
        "UID": UID,
        "FieldID": FieldID,
    }

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response


FieldID = "1718261488045"
sensed_days = get_sesnsed_days(UID, FieldID)
sensed_days.json()

# Get Sat Image / Image URl


In [ ]:
def get_sat_image(UID, fieldId, SensedDay, ImageType):

    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getFieldImage"
    bodyObj = {
        "UID": UID,
        "FieldID": fieldId,
        "ImageType": ImageType,
        "SensedDay": SensedDay,
        "Colormap": "1",
    }

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response.json()["url"]


FieldID = "1718261488045"
SensedDay = "20240611"
ImageType = "ndvi"  # keep it lowercase
get_sat_image(UID, FieldID, SensedDay, ImageType)